# Generate Virtual Cohort

Written by: Anna Qi

Example code to visualize some meshes regenerated from shape mode scores.

In [ ]:
# imports (use pip install in your terminal to install these packages)
import pandas as pd
import pyvista as pv
import scipy.io
import numpy as np

In [19]:
import scipy.io

# Load UKB EDES atlas from .mat file
mat_data = scipy.io.loadmat("Data/BioBank_EDES_200.mat") #Replace with the actual path to your library folder
pca = mat_data['pca200'][0, 0]

# functions to reconstruct shape given PC scores
def reconstruct_shape(score, atlas, num_scores = 25):
    d = score * np.sqrt(atlas["LATENT"][0:num_scores]).T
    shape = atlas["MU"] + np.matmul(d, atlas["COEFF"][:, :num_scores].T)
    return shape.T

# Extract ED phase as (N, 3) mesh
def get_ED_mesh_from_shape(shape):
    N = len(shape)
    return shape[:N // 2].reshape(-1, 3)

def get_ES_mesh_from_shape(shape):
    N = len(shape)
    return shape[N // 2:].reshape(-1, 3)

In [20]:
# import necessary ET Indices File
et_indices = np.loadtxt("biv-me-main/src/model/ETIndicesSorted.txt").astype(int) - 1 #matlab indexing to python

# Import virtual cohort sheet
virtual_cohort = pd.read_excel("Data/virtual_cohort_data.xlsx")
pc_columns = [col for col in virtual_cohort.columns if col.startswith('PC')]
pc_scores = virtual_cohort[pc_columns]

### Visualize some individual data

In [15]:
# visualize shape from the z-scores
pv.set_jupyter_backend('trame')

# Surface region index ranges
SurfaceStartEnd = [
    (0, 3072),     # LV
    (3072, 4480),  # RVS
    (4480, 6752),  # RVFW
    (6752, 11616), # Epi
    (11616, 11664),# Mitral
    (11664, 11688),# Aortic
    (11688, 11728),# Tricuspid
    (11728, 11760) # Pulmonary
]

# define functions to visualize a mesh using pyvista
def build_pv_mesh(vertices, faces):
    # Convert to pyvista face format: [3, i0, i1, i2] per triangle
    n_faces = faces.shape[0]
    face_data = np.hstack([np.full((n_faces, 1), 3), faces]).astype(np.int32)
    return pv.PolyData(vertices, face_data)

def plot_ed_shape_pyvista(vertices, et_indices, show_valves=True):
    # Segment faces
    LV_faces = et_indices[SurfaceStartEnd[0][0]:SurfaceStartEnd[0][1]]
    RV_faces = et_indices[SurfaceStartEnd[1][0]:SurfaceStartEnd[2][1]]
    EPI_faces = et_indices[SurfaceStartEnd[3][0]:SurfaceStartEnd[3][1]]

    LV_mesh = build_pv_mesh(vertices, LV_faces)
    RV_mesh = build_pv_mesh(vertices, RV_faces)
    EPI_mesh = build_pv_mesh(vertices, EPI_faces)

    plotter = pv.Plotter(notebook=True)
    plotter.set_background("white")

    # Add surfaces
    plotter.add_mesh(LV_mesh, color=(0, 0.5, 0), show_edges=False, smooth_shading=True, opacity=1.0, name="LV")
    plotter.add_mesh(RV_mesh, color=(0, 0, 0.8), show_edges=False, smooth_shading=True, opacity=1.0, name="RV")
    plotter.add_mesh(EPI_mesh, color=(0.5, 0, 0), show_edges=False, smooth_shading=True, opacity=0.5, name="Epi")

    if show_valves:
        _add_valves_to_plotter(plotter, vertices, et_indices)

    plotter.view_vector((1, 1, 0.5), viewup=(0, 0, 1))
    return plotter.show()

# add valves to the plot
def _add_valves_to_plotter(plotter, vertices, et_indices):
    def valve_lines(start, end, color):
        faces = et_indices[start:end]
        for i in range(0, len(faces), 2):
            try:
                v1 = vertices[faces[i, [0, 2]]]
                v2 = vertices[faces[i + 1, [0, 1]]]
                points = np.vstack([v1, v2])
                line = pv.lines_from_points(points)
                plotter.add_mesh(line, color=color, line_width=2)
            except IndexError:
                continue

    valve_lines(*SurfaceStartEnd[4], 'cyan')     # Mitral
    valve_lines(*SurfaceStartEnd[5], 'yellow')   # Aortic
    valve_lines(*SurfaceStartEnd[6], 'magenta')  # Tricuspid
    valve_lines(*SurfaceStartEnd[7], 'green')    # Pulmonary

In [16]:
patient1 = pc_scores.loc[0].to_numpy()
patient1_shape = reconstruct_shape(patient1, pca)
patient1_ed = get_ED_mesh_from_shape(patient1_shape)

plot_ed_shape_pyvista(patient1_ed, et_indices)

Widget(value='<iframe src="http://localhost:51177/index.html?ui=P_0x13614d550_2&reconnect=auto" class="pyvista…

In [17]:
# modified plotting function to fit in a subplot
def _add_model_to_subplot(plotter, vertices, et_indices):
    LV_faces = et_indices[SurfaceStartEnd[0][0]:SurfaceStartEnd[0][1]]
    RV_faces = et_indices[SurfaceStartEnd[1][0]:SurfaceStartEnd[2][1]]
    EPI_faces = et_indices[SurfaceStartEnd[3][0]:SurfaceStartEnd[3][1]]

    LV_mesh = build_pv_mesh(vertices, LV_faces)
    RV_mesh = build_pv_mesh(vertices, RV_faces)
    EPI_mesh = build_pv_mesh(vertices, EPI_faces)

    plotter.add_mesh(LV_mesh, color=(0, 0.5, 0), show_edges=False, smooth_shading=True, name="LV")
    plotter.add_mesh(RV_mesh, color=(0, 0, 0.8), show_edges=False, smooth_shading=True, name="RV")
    plotter.add_mesh(EPI_mesh, color=(0.5, 0, 0), show_edges=False, smooth_shading=True, opacity=0.5, name="Epi")

# function plot two models side-by-side with linked cameras
def plot_comparison(vertices1, label1, vertices2, label2, et_indices):
    # create a plotter with 1 row and 2 columns
    plotter = pv.Plotter(shape=(1, 2), notebook=True, window_size=[1800, 700])
    plotter.set_background("white")

    # add first model on the left
    plotter.subplot(0, 0)
    plotter.add_text(label1, font_size=24, color='black')
    _add_model_to_subplot(plotter, vertices1, et_indices)
    plotter.view_vector((1, 1, 0.5), viewup=(0, 0, 1))

    # add the second model on the right
    plotter.subplot(0, 1)
    plotter.add_text(label2, font_size=24, color='black')
    _add_model_to_subplot(plotter, vertices2, et_indices)
    plotter.view_vector((1, 1, 0.5), viewup=(0, 0, 1))

    # link the cameras
    plotter.link_views()
    print("Displaying two models. Rotate one and the other will follow.")
    plotter.show()

et_indices = np.loadtxt("biv-me-main/src/model/ETIndicesSorted.txt").astype(int) - 1

patient1 = pc_scores.loc[0].to_numpy()
patient1_shape = reconstruct_shape(patient1, pca)
patient1_ed = get_ED_mesh_from_shape(patient1_shape)

patient2 = pc_scores.loc[1].to_numpy()
patient2_shape = reconstruct_shape(patient2, pca)
patient2_ed = get_ED_mesh_from_shape(patient2_shape)

# Call the plotting function
plot_comparison(
    vertices1=patient1_ed, label1="Patient 1",
    vertices2=patient2_ed, label2="Patient 2",
    et_indices=et_indices
)

Displaying two models. Rotate one and the other will follow.


Widget(value='<iframe src="http://localhost:51177/index.html?ui=P_0x135890ce0_3&reconnect=auto" class="pyvista…